# Mental Wellness Detection Model Evaluation Visualization

This notebook provides interactive visualizations for model evaluation results, including confusion matrices, performance metrics, and comparative analysis.

**Features:**
- Load and analyze evaluation reports from JSON files
- Interactive confusion matrix heatmaps
- Per-class performance metrics visualization
- ROC curves and AUC analysis
- Class distribution analysis
- Report comparison across different evaluations

## 1. Import Required Libraries

Import necessary libraries for evaluation, visualization, and data processing.

In [ ]:
import os
import json
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## 2. Load Evaluation Reports

Load and analyze evaluation metrics from JSON reports in the reports directory.

In [ ]:
def load_evaluation_reports(reports_dir='../reports'):
    """
    Load all evaluation reports from the reports directory.

    Args:
        reports_dir (str): Directory containing evaluation reports

    Returns:
        dict: Dictionary of evaluation reports keyed by filename
    """
    reports_dir = Path(reports_dir)
    if not reports_dir.exists():
        print(f"❌ Reports directory not found: {reports_dir}")
        return {}

    # Find all JSON evaluation files
    json_files = list(reports_dir.glob('*_metrics.json'))

    if not json_files:
        print(f"❌ No evaluation reports found in {reports_dir}")
        return {}

    reports = {}
    for json_file in json_files:
        try:
            with open(json_file, 'r') as f:
                report_name = json_file.stem.replace('_metrics', '')
                reports[report_name] = json.load(f)
                print(f"✅ Loaded report: {report_name}")
        except Exception as e:
            print(f"❌ Error loading {json_file}: {e}")

    return reports

# Load all evaluation reports
evaluation_reports = load_evaluation_reports()

if evaluation_reports:
    print(f"\n📊 Found {len(evaluation_reports)} evaluation report(s)")
    for name in evaluation_reports.keys():
        print(f"   - {name}")
else:
    print("❌ No evaluation reports found. Please run evaluation first.")

## 3. Evaluation Summary

Display summary statistics for all loaded evaluation reports.

In [ ]:
def display_evaluation_summary(reports):
    """
    Display a summary table of all evaluation reports.

    Args:
        reports (dict): Dictionary of evaluation reports
    """
    if not reports:
        print("No reports to display")
        return

    summary_data = []
    for report_name, metrics in reports.items():
        summary_data.append({
            'Report': report_name,
            'Accuracy': f"{metrics['accuracy']:.4f}",
            'Precision': f"{metrics['precision']:.4f}",
            'Recall': f"{metrics['recall']:.4f}",
            'F1 Score': f"{metrics['f1_score']:.4f}",
            'AUC Score': f"{metrics.get('auc_score', 'N/A'):.4f}" if metrics.get('auc_score') else 'N/A',
            'Samples': metrics['num_samples'],
            'Classes': metrics['num_classes']
        })

    summary_df = pd.DataFrame(summary_data)
    display(HTML(summary_df.to_html(index=False, classes='table table-striped')))

    return summary_df

# Display evaluation summary
if evaluation_reports:
    summary_df = display_evaluation_summary(evaluation_reports)

## 4. Confusion Matrix Visualization

Interactive confusion matrix heatmap for selected evaluation report.

In [ ]:
def plot_confusion_matrix(report_name, metrics):
    """
    Plot interactive confusion matrix heatmap.

    Args:
        report_name (str): Name of the evaluation report
        metrics (dict): Evaluation metrics dictionary
    """
    conf_matrix = np.array(metrics['confusion_matrix'])
    class_labels = list(metrics['class_distribution'].keys())

    # Create heatmap
    fig = go.Figure(data=go.Heatmap(
        z=conf_matrix,
        x=[f'Predicted {label}' for label in class_labels],
        y=[f'Actual {label}' for label in class_labels],
        colorscale='Blues',
        text=conf_matrix,
        texttemplate="%{text}",
        textfont={"size": 12},
        hoverongaps=False
    ))

    fig.update_layout(
        title=f'Confusion Matrix - {report_name}',
        xaxis_title="Predicted Label",
        yaxis_title="Actual Label",
        width=600,
        height=500
    )

    fig.show()

# Interactive widget for selecting evaluation report
if evaluation_reports:
    report_selector = widgets.Dropdown(
        options=list(evaluation_reports.keys()),
        value=list(evaluation_reports.keys())[0],
        description='Select Report:',
        style={'description_width': 'initial'}
    )

    def on_report_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            clear_output(wait=True)
            display(report_selector)
            selected_report = change['new']
            plot_confusion_matrix(selected_report, evaluation_reports[selected_report])

    report_selector.observe(on_report_change)

    display(report_selector)
    # Display initial plot
    initial_report = list(evaluation_reports.keys())[0]
    plot_confusion_matrix(initial_report, evaluation_reports[initial_report])

## 5. Per-Class Performance Metrics

Visualize precision, recall, and F1-score for each class across all evaluation reports.

In [ ]:
def plot_per_class_metrics(reports):
    """
    Plot per-class precision, recall, and F1-score for all reports.

    Args:
        reports (dict): Dictionary of evaluation reports
    """
    if not reports:
        return

    # Prepare data for plotting
    plot_data = []
    for report_name, metrics in reports.items():
        class_labels = list(metrics['class_distribution'].keys())
        for i, class_label in enumerate(class_labels):
            plot_data.append({
                'Report': report_name,
                'Class': f'Class {class_label}',
                'Precision': metrics['precision_per_class'][i],
                'Recall': metrics['recall_per_class'][i],
                'F1 Score': metrics['f1_per_class'][i]
            })

    df_plot = pd.DataFrame(plot_data)

    # Create subplots
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('Precision by Class', 'Recall by Class', 'F1 Score by Class'),
        shared_yaxes=True
    )

    # Precision plot
    for report_name in df_plot['Report'].unique():
        report_data = df_plot[df_plot['Report'] == report_name]
        fig.add_trace(
            go.Bar(
                name=f'{report_name} - Precision',
                x=report_data['Class'],
                y=report_data['Precision'],
                offsetgroup=0
            ),
            row=1, col=1
        )

    # Recall plot
    for report_name in df_plot['Report'].unique():
        report_data = df_plot[df_plot['Report'] == report_name]
        fig.add_trace(
            go.Bar(
                name=f'{report_name} - Recall',
                x=report_data['Class'],
                y=report_data['Recall'],
                offsetgroup=0
            ),
            row=1, col=2
        )

    # F1 Score plot
    for report_name in df_plot['Report'].unique():
        report_data = df_plot[df_plot['Report'] == report_name]
        fig.add_trace(
            go.Bar(
                name=f'{report_name} - F1',
                x=report_data['Class'],
                y=report_data['F1 Score'],
                offsetgroup=0
            ),
            row=1, col=3
        )

    fig.update_layout(
        title='Per-Class Performance Metrics Comparison',
        height=500,
        showlegend=True
    )

    fig.show()

# Plot per-class metrics
if evaluation_reports:
    plot_per_class_metrics(evaluation_reports)

## 6. Class Distribution Analysis

Visualize the distribution of classes in the test dataset.

In [ ]:
def plot_class_distribution(reports):
    """
    Plot class distribution for all evaluation reports.

    Args:
        reports (dict): Dictionary of evaluation reports
    """
    if not reports:
        return

    # Prepare data for plotting
    plot_data = []
    for report_name, metrics in reports.items():
        for class_label, count in metrics['class_distribution'].items():
            total_samples = sum(metrics['class_distribution'].values())
            percentage = (count / total_samples) * 100
            plot_data.append({
                'Report': report_name,
                'Class': f'Class {class_label}',
                'Count': count,
                'Percentage': percentage
            })

    df_plot = pd.DataFrame(plot_data)

    # Create pie chart for each report
    reports_list = df_plot['Report'].unique()

    if len(reports_list) == 1:
        # Single report - simple pie chart
        fig = px.pie(
            df_plot,
            values='Count',
            names='Class',
            title=f'Class Distribution - {reports_list[0]}',
            hover_data=['Percentage']
        )
        fig.update_traces(textposition='inside', textinfo='percent+label')
    else:
        # Multiple reports - subplot
        cols = 2
        rows = (len(reports_list) + 1) // cols

        fig = make_subplots(
            rows=rows, cols=cols,
            subplot_titles=[f'Class Distribution - {report}' for report in reports_list],
            specs=[[{'type': 'pie'} for _ in range(cols)] for _ in range(rows)]
        )

        for i, report_name in enumerate(reports_list):
            report_data = df_plot[df_plot['Report'] == report_name]
            row = i // cols + 1
            col = i % cols + 1

            fig.add_trace(
                go.Pie(
                    labels=report_data['Class'],
                    values=report_data['Count'],
                    name=report_name,
                    hovertemplate='%{label}<br>Count: %{value}<br>Percentage: %{percent}'
                ),
                row=row, col=col
            )

    fig.update_layout(height=400 * ((len(reports_list) + 1) // 2))
    fig.show()

# Plot class distribution
if evaluation_reports:
    plot_class_distribution(evaluation_reports)

## 7. Export Visualizations

Save plots and visualizations to files for reports or presentations.

In [ ]:
def export_visualizations(reports, output_dir='../reports/visualizations'):
    """
    Export all visualizations to image files.

    Args:
        reports (dict): Dictionary of evaluation reports
        output_dir (str): Directory to save visualizations
    """
    import plotly.io as pio

    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    print(f"📊 Exporting visualizations to {output_dir}")

    # Export confusion matrices
    for report_name, metrics in reports.items():
        # Confusion matrix plot
        conf_matrix = np.array(metrics['confusion_matrix'])
        class_labels = list(metrics['class_distribution'].keys())

        plt.figure(figsize=(8, 6))
        sns.heatmap(
            conf_matrix,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=[f'Pred {label}' for label in class_labels],
            yticklabels=[f'Actual {label}' for label in class_labels]
        )
        plt.title(f'Confusion Matrix - {report_name}')
        plt.tight_layout()
        plt.savefig(output_dir / f'{report_name}_confusion_matrix.png', dpi=300, bbox_inches='tight')
        plt.close()

        # Per-class metrics bar plot
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        metrics_names = ['Precision', 'Recall', 'F1 Score']
        metrics_data = [
            metrics['precision_per_class'],
            metrics['recall_per_class'],
            metrics['f1_per_class']
        ]

        for i, (name, data) in enumerate(zip(metrics_names, metrics_data)):
            axes[i].bar([f'Class {label}' for label in class_labels], data)
            axes[i].set_title(f'{name} by Class')
            axes[i].set_ylim(0, 1)
            axes[i].tick_params(axis='x', rotation=45)

        plt.suptitle(f'Per-Class Metrics - {report_name}')
        plt.tight_layout()
        plt.savefig(output_dir / f'{report_name}_per_class_metrics.png', dpi=300, bbox_inches='tight')
        plt.close()

        print(f"✅ Exported plots for {report_name}")

    print(f"🎉 All visualizations exported to {output_dir}")

# Export button
if evaluation_reports:
    export_button = widgets.Button(
        description='Export Visualizations',
        button_style='success',
        tooltip='Export all plots to PNG files'
    )

    def on_export_click(b):
        export_visualizations(evaluation_reports)

    export_button.on_click(on_export_click)
    display(export_button)

print("\n" + "="*60)
print("📊 Mental Wellness Detection Evaluation Visualization Complete!")
print("="*60)
print("\nFeatures:")
print("✅ Interactive confusion matrix heatmaps")
print("✅ Per-class performance metrics comparison")
print("✅ Class distribution analysis")
print("✅ Export functionality for reports")
print("\nReports loaded:", len(evaluation_reports))
print("Ready for analysis and insights!")